# Trading

- https://github.com/kelvinau/crypto-arbitrage : Arbitrage, not pip installable, not many exchanges
- https://github.com/ScottFreeLLC/AlphaPy : Machine learning framework for prediction
- https://github.com/ssatia/cointreau : LSTM trading model, not pip instalable, use for a workflow example
- https://github.com/Endogen/Telegram-Kraken-Bot : Interactive trading bot through Telegram
- https://github.com/bmoscon/cryptofeed : Normalizes prices across exchanges but doesn't use ccxt.
- https://github.com/samre12/gym-cryptotrading : Uses AI gym and reinforcement learning
- https://github.com/meister245/CryptoLunch : Find all arbitrage oppurtunities from coinmarketcap listed exchanges for all coins
- https://github.com/owocki/pytrader : Machine learning trading platform


## Interesting

- https://github.com/gcarq/freqtrade : Telegram trading bot
- https://github.com/ccxt/ccxt : Exchange API Library
- https://github.com/Denton24646/Titan : Trading Framework but not pip installable. Good wiki on structure: https://github.com/Denton24646/Titan/wiki Scavenge?
- https://github.com/CryptoSignal/crypto-signal : Platform for signals, again not pip installable. Uses ccxt, plugs into telegram and others. Scavenge?
- https://github.com/ScottFreeLLC/AlphaPy : Machine learning framework for prediction
- https://github.com/blue-yonder/tsfresh : Automatic time series feature extraction


## Looking for

- Speech recognition type graph analysis
